
## Check GMF integrity


In [1]:
import os
import cv2
import numpy as np
import time
start = time.time()
os.system("python inference.py")
end = time.time()
print("Time elapsed: ", end - start)

# read vid_out/0000001.png and ref.png and compare them with np
out = cv2.imread("check.png")
ref = cv2.imread("ref.png")
np.allclose(out, ref, atol=1, equal_nan=True)

Time elapsed:  11.203269720077515


True

## gen pt

In [1]:
import torch
from gmflow.gmflow import GMFlow
import os
import cv2

device = torch.device("cuda")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

_image_path = r'.'
shape = (960, 544)  # override the shape variable above, indent if necessary
shape_t = (shape[1], shape[0])
_i0 = cv2.resize(cv2.imread(os.path.join(_image_path, r'0022.jpg')), shape)
_i1 = cv2.resize(cv2.imread(os.path.join(_image_path, r'0023.jpg')), shape)
i0 = torch.from_numpy(_i0).unsqueeze(0).permute(0,3,1,2) / 255.
i1 = torch.from_numpy(_i1).unsqueeze(0).permute(0,3,1,2) / 255.
bs = 1

flownet = GMFlow()
flownet.load_state_dict(torch.load('flownet.pkl'))



c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<All keys matched successfully>

In [4]:
model = torch.jit.trace(flownet, (i0, i1))
torch.jit.save(model, "flownet_544.pt")

e:\onnx\GMFSS2NCNN\gmflow\utils.py:58: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1).to(img1.device)
e:\onnx\GMFSS2NCNN\gmflow\utils.py:59: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1).to(img1.device)
e:\onnx\GMFSS2NCNN\gmflow\utils.py:21: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flo

# Compare for Mark

# Reference

In [6]:
import torch
import torchvision.models as models

net = models.resnet18(pretrained=False)
net = net.eval()

x = torch.rand(1, 3, 224, 224)

# You could try disabling checking when tracing raises error
# mod = torch.jit.trace(net, x, check_trace=False)
mod = torch.jit.trace(net, x)

mod.save("resnet18.pt")

c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Administrator\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
